

Import packages


In [19]:
import requests
import pandas as pd
import json
import math

Load datasets - matches + all players ratings by match


In [41]:
df_matches = pd.read_csv('fotmob-premier-league-matches-results.csv')
df_matches_players_stats = pd.read_csv('fotmob-premier-league-matches-players-stats.csv')

In [42]:
df_matches_players_unique = df_matches_players_stats['player_id'].unique()
len(df_matches_players_unique)

990

In [43]:
# create list of all players with their id 
dict_players = {}
for _player_id in df_matches_players_unique:
  if math.isnan(_player_id) == False:
    dict_players['home_player_' + str(round(_player_id))] = 0
    dict_players['away_player_' + str(round(_player_id))] = 0
len(dict_players)

1980

In [75]:


data_match_lineup_rating = []
for i, _match in df_matches.T.iteritems():
  if _match['match_started'] == True and _match['match_finished'] == True:

    # classify Win Loose Draw
    _match['match_home_result'] = 'D'
    _match['match_away_result'] = 'D'
    if _match['match_home_team_score'] > _match['match_away_team_score']:
      _match['match_home_result'] = 'W'
      _match['match_away_result'] = 'L'
    else:
      _match['match_home_result'] = 'L'
      _match['match_away_result'] = 'W'

    # get players played during matches
    _players = df_matches_players_stats[df_matches_players_stats['match_id'] == _match['match_id']]
    _players_line_up = dict_players.copy()
    for i_player, _player in _players.T.iteritems():
      if _player['player_team_id'] == _match['match_home_team_id']:
        _players_line_up['home_player_' + str(round(_player['player_id']))] = _player['player_rating']
      elif _player['player_team_id'] == _match['match_away_team_id']:
        _players_line_up['away_player_' + str(round(_player['player_id']))] = _player['player_rating']

    # build new row
    _m = {
      'match_id' : _match['match_id'],
      'match_utc' : _match['match_time_utc_date'],
      'home_team_id' : _match['match_home_team_id'],
      'home_team_name' : _match['match_home_team_name'],
      'home_team_score' : _match['match_home_team_score'],
      'home_team_result' : _match['match_home_result'],
      'away_team_id' : _match['match_away_team_id'],
      'away_team_name' : _match['match_away_team_name'],
      'away_team_score' : _match['match_away_team_score'],
      'away_team_result' : _match['match_away_result'],
    }

    # add players lineup at each row
    _m.update(_players_line_up)
    data_match_lineup_rating.append(_m)

df_match_lineup_rating = pd.DataFrame(data_match_lineup_rating)
df_match_lineup_rating.to_csv('fotmob-premier-league-matches-lineup.csv', index=False, sep=';')

Build model classification :

In [77]:
df_match_lineup_rating.head()

,match_id,match_utc,home_team_id,home_team_name,home_team_score,home_team_result,away_team_id,away_team_name,away_team_score,away_team_result,...,home_player_131991,away_player_131991,home_player_660321,away_player_660321,home_player_719219,away_player_719219,home_player_244811,away_player_244811,home_player_478367,away_player_478367
0,3900932,2022-08-05T19:00:00.000Z,9826,Crystal Palace,0,L,9825,Arsenal,2,W,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
1,3900933,2022-08-06T11:30:00.000Z,9879,Fulham,2,L,8650,Liverpool,2,W,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
2,3900934,2022-08-06T14:00:00.000Z,8678,AFC Bournemouth,2,W,10252,Aston Villa,0,L,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
3,3900935,2022-08-06T14:00:00.000Z,8463,Leeds United,2,W,8602,Wolverhampton Wanderers,1,L,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
4,3900937,2022-08-06T14:00:00.000Z,10261,Newcastle United,2,W,10203,Nottingham Forest,0,L,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0


Encoding

In [80]:
from sklearn.preprocessing import OrdinalEncoder
ordinal_encoder = OrdinalEncoder(categories = [["L","D","W"]])
ordinal_encoder.fit(df_match_lineup_rating[["home_team_result"]])
df_match_lineup_rating["home_team_result_encoded"] = ordinal_encoder.transform(df_match_lineup_rating[["home_team_result"]])


1790
1766


Stochastic Gradient Descent SGD



In [162]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import SGDClassifier

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Model
clf_SGD = SGDClassifier(loss='hinge', penalty='l2', alpha=0.0001, max_iter=1000, tol=1e-3)
clf_SGD.fit(X_train, y_train)

# predict and accuracy
y_pred_SGD = clf_SGD.predict(X_test)
acc_SGD = accuracy_score(y_test, y_pred_SGD)
print("Accuracy: ", acc_SGD)

Accuracy:  0.5150943396226415


LinearSVC

In [163]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import LinearSVC

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
clf_SVC = LinearSVC(max_iter=20000)
clf_SVC.fit(X_train, y_train)

# predict and accuracy
y_pred_SVC = clf_SVC.predict(X_test)
acc_SVC = accuracy_score(y_test, y_pred_SVC)
print("Accuracy: ", acc_SVC)

Accuracy:  0.46226415094339623


/usr/local/lib/python3.9/dist-packages/sklearn/svm/_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


Logistic Regression

In [164]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
log_reg = LogisticRegression(solver='lbfgs',max_iter=10000)
log_reg.fit(X_train, y_train)

# predict and accuracy
y_pred = log_reg.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)

print("Accuracy:", accuracy)

Accuracy: 0.6962264150943396


Naives Bayes

In [165]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
clf_nb = GaussianNB()
clf_nb.fit(X_train, y_train)

# predict and accuracy
y_pred = clf_nb.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6226415094339622


K-Nearest Neighbours

In [166]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
clf_knn = KNeighborsClassifier(n_neighbors=3)
clf_knn.fit(X_train, y_train)

# predict and accuracy
y_pred = clf_knn.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5735849056603773


Decision Tree

In [167]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
clf_dt = DecisionTreeClassifier()
clf_dt.fit(X_train, y_train)

# predict and accuracy
y_pred = clf_dt.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.7245283018867924


Random Forest

In [173]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
clf_rf = RandomForestClassifier(n_estimators=100, random_state=42)
clf_rf.fit(X_train, y_train)

# predict and accuracy
y_pred = clf_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.8415094339622642


Support Vector Machine

In [169]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# model
clf_svm = SVC(kernel='linear', C=1, random_state=42)
clf_svm.fit(X_train, y_train)

# predict and accuracy
y_pred = clf_svm.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.6830188679245283


In [170]:
X_test

,home_team_id,away_team_id,home_player_109222,away_player_109222,home_player_157838,away_player_157838,home_player_503659,away_player_503659,home_player_844425,away_player_844425,...,home_player_131991,away_player_131991,home_player_660321,away_player_660321,home_player_719219,away_player_719219,home_player_244811,away_player_244811,home_player_478367,away_player_478367
141,8654,8197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
983,10261,9825,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
609,8586,8197,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
484,8191,9817,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
1715,10204,8466,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1015,8668,8602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
755,8650,8602,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
1109,10261,10260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0
1369,10252,10260,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0,0.0,0.0,0.0,0.0,0,0.0,0,0.0


Fine tune Random Forest

In [171]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score

# X, Y and split
df_match_lineup_rating = df_match_lineup_rating.dropna()
X = df_match_lineup_rating.drop(['home_team_result_encoded', 'match_id', 'match_utc', 'home_team_name', 'home_team_score', 'home_team_result', 'away_team_name', 'away_team_score', 'away_team_result'], axis=1)
y = df_match_lineup_rating['home_team_result_encoded']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

# Define hyperparameters
params = {
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# Randomized Search CV
rf = RandomForestClassifier(random_state=42)
clf_rf = RandomizedSearchCV(rf, params, n_iter=50, cv=5, random_state=42)
clf_rf.fit(X_train, y_train)

# Best parameters and accuracy
best_params = clf_rf.best_params_
print("Best parameters:", best_params)
y_pred = clf_rf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Best parameters: {'n_estimators': 50, 'min_samples_split': 10, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': True}
Accuracy: 0.8301886792452831
